In [1]:
import pandas as pd
df=pd.read_csv(r'ready_file.csv')

In [2]:
df.drop(columns='adj close', inplace=True)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 609043 entries, 0 to 609042
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   date                609043 non-null  object 
 1   open                609043 non-null  float64
 2   high                609043 non-null  float64
 3   low                 609043 non-null  float64
 4   close               609043 non-null  float64
 5   volume              609043 non-null  int64  
 6   dividends           609043 non-null  float64
 7   stock splits        609043 non-null  float64
 8   symbol              609043 non-null  object 
 9   capital gains       609043 non-null  float64
 10  previous day close  609043 non-null  float64
dtypes: float64(8), int64(1), object(2)
memory usage: 51.1+ MB


In [4]:
import pandas as pd

# Define the rolling period as a variable
rolling_period = 15

# Ensure 'date' is in datetime format
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'])
else:
    raise KeyError("The 'date' column is missing in the DataFrame.")

# Set the date as index for rolling calculations (if needed)
df.set_index('date', inplace=True)

# Function to calculate adjusted close with a customizable rolling period
def calculate_adjusted_close(group, period=rolling_period):
    # Calculate rolling sums for dividends and stock splits over the specified period
    group['rolling_dividends'] = group['dividends'].rolling(window=period).sum()
    group['rolling_splits'] = group['stock splits'].rolling(window=period).sum()
    
    # Calculate adjusted close based on rolling dividends and splits
    group['adj close'] = group['close'] + group['rolling_dividends'] - group['rolling_splits']
    
    return group

# Group by 'symbol' and apply the adjusted close calculation
df = df.groupby('symbol', group_keys=False).apply(calculate_adjusted_close, period=rolling_period)

# Reset the index to bring 'date' back as a column
df.reset_index(inplace=True)

# Create a column for previous day's adjusted close
df['prev_close'] = df.groupby('symbol')['adj close'].shift(1)

# Optional: Sort the DataFrame by date and symbol for better readability
df.sort_values(by=['symbol', 'date'], inplace=True)

# Display the first few rows of the updated DataFrame
print(df[['date', 'symbol', 'close', 'dividends', 'adj close', 'prev_close']].head(20))


                            date symbol       close  dividends   adj close  \
0      2024-06-09 20:00:00-04:00      A  132.783569      0.000         NaN   
9785   2024-06-10 20:00:00-04:00      A  131.865280      0.000         NaN   
19571  2024-06-11 20:00:00-04:00      A  132.913330      0.000         NaN   
29361  2024-06-12 20:00:00-04:00      A  130.328125      0.000         NaN   
39154  2024-06-13 20:00:00-04:00      A  129.609467      0.000         NaN   
48951  2024-06-16 20:00:00-04:00      A  131.585800      0.000         NaN   
58754  2024-06-17 20:00:00-04:00      A  134.650101      0.000         NaN   
68560  2024-06-19 20:00:00-04:00      A  132.484116      0.000         NaN   
78371  2024-06-20 20:00:00-04:00      A  133.003159      0.000         NaN   
88183  2024-06-23 20:00:00-04:00      A  134.829773      0.000         NaN   
97999  2024-06-24 20:00:00-04:00      A  134.440491      0.000         NaN   
107822 2024-06-25 20:00:00-04:00      A  132.843460      0.000  

In [6]:
#Lets create two new features maximum day differing and difference of open and close price.
df ['maxdiff'] = df[ 'high'] - df['low']
df ['daydiff'] = df[ 'close'] - df['open']

In [7]:
df.head(50)

,date,open,high,low,close,volume,dividends,stock splits,symbol,capital gains,previous day close,rolling_dividends,rolling_splits,adj close,prev_close,maxdiff,daydiff
0,2024-06-09 20:00:00-04:00,132.623862,133.602058,132.005015,132.783569,1673600,0.000,1.0,A,0.049911,132.733658,NaN,NaN,NaN,NaN,1.597042,0.159707
9785,2024-06-10 20:00:00-04:00,132.394297,132.743655,131.246434,131.865280,2081600,0.000,1.0,A,-0.918289,132.783569,NaN,NaN,NaN,NaN,1.497221,-0.529017
19571,2024-06-11 20:00:00-04:00,133.382461,133.392437,132.084866,132.913330,2399900,0.000,1.0,A,1.048050,131.865280,NaN,NaN,NaN,NaN,1.307571,-0.469131
29361,2024-06-12 20:00:00-04:00,132.344372,132.753616,129.799093,130.328125,2555400,0.000,1.0,A,-2.585205,132.913330,NaN,NaN,NaN,NaN,2.954523,-2.016247
39154,2024-06-13 20:00:00-04:00,129.759183,130.497817,129.000596,129.609467,2163300,0.000,1.0,A,-0.718658,130.328125,NaN,NaN,NaN,NaN,1.497221,-0.149716
48951,2024-06-16 20:00:00-04:00,130.867133,131.825361,129.260114,131.585800,2090500,0.000,1.0,A,1.976334,129.609467,NaN,NaN,NaN,NaN,2.565247,0.718667
58754,2024-06-17 20:00:00-04:00,131.885243,134.899638,131.316291,134.650101,3453000,0.000,1.0,A,3.064301,131.585800,NaN,NaN,NaN,NaN,3.583346,2.764858
68560,2024-06-19 20:00:00-04:00,133.013133,134.400557,132.284489,132.484116,2887400,0.000,1.0,A,-2.165985,134.650101,NaN,NaN,NaN,NaN,2.116068,-0.529017
78371,2024-06-20 20:00:00-04:00,132.603906,134.250843,131.955101,133.003159,4461300,0.000,1.0,A,0.519043,132.484116,NaN,NaN,NaN,NaN,2.295742,0.399253
88183,2024-06-23 20:00:00-04:00,135.049367,137.454906,134.091138,134.829773,3339500,0.000,1.0,A,1.826614,133.003159,NaN,NaN,NaN,NaN,3.363768,-0.219594


In [8]:
import numpy as np
import pandas as pd

def RSI(series, period):
    # Ensure the input series is a float
    series = series.astype(float)
    
    # Calculate price changes
    delta = series.diff()
    
    # Calculate gains and losses
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    
    # Calculate the relative strength (RS) and the RSI
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    
    # Return the RSI, aligning with the original series
    return rsi

# Assuming df is your original DataFrame and it has 'symbol' and 'adj close' columns
rsi_period = 10

# Initialize the RSI column with NaN
df['RSI'] = np.nan  

# Group by 'symbol' and calculate RSI for each group
for symbol, group in df.groupby('symbol'):
    df.loc[group.index, 'RSI'] = RSI(group['adj close'], rsi_period)

# Display the DataFrame with the newly added RSI
print(df.head())


                           date        open        high         low  \
0     2024-06-09 20:00:00-04:00  132.623862  133.602058  132.005015   
9785  2024-06-10 20:00:00-04:00  132.394297  132.743655  131.246434   
19571 2024-06-11 20:00:00-04:00  133.382461  133.392437  132.084866   
29361 2024-06-12 20:00:00-04:00  132.344372  132.753616  129.799093   
39154 2024-06-13 20:00:00-04:00  129.759183  130.497817  129.000596   

            close   volume  dividends  stock splits symbol  capital gains  \
0      132.783569  1673600        0.0           1.0      A       0.049911   
9785   131.865280  2081600        0.0           1.0      A      -0.918289   
19571  132.913330  2399900        0.0           1.0      A       1.048050   
29361  130.328125  2555400        0.0           1.0      A      -2.585205   
39154  129.609467  2163300        0.0           1.0      A      -0.718658   

       previous day close  rolling_dividends  rolling_splits  adj close  \
0              132.733658          

In [9]:
def aroon(data, lb=15):
    df = data.copy()
    df['up'] = 100 * df['high'].rolling(lb + 1).apply(lambda x: x.argmax()) / lb
    df['dn'] = 100 * df['low'].rolling(lb + 1).apply(lambda x: x.argmin()) / lb
    return df['up'], df['dn']
df['up'],df['dn'] = aroon(df, lb=15)

In [10]:
df.head(50)

,date,open,high,low,close,volume,dividends,stock splits,symbol,capital gains,previous day close,rolling_dividends,rolling_splits,adj close,prev_close,maxdiff,daydiff,RSI,up,dn
0,2024-06-09 20:00:00-04:00,132.623862,133.602058,132.005015,132.783569,1673600,0.000,1.0,A,0.049911,132.733658,NaN,NaN,NaN,NaN,1.597042,0.159707,NaN,NaN,NaN
9785,2024-06-10 20:00:00-04:00,132.394297,132.743655,131.246434,131.865280,2081600,0.000,1.0,A,-0.918289,132.783569,NaN,NaN,NaN,NaN,1.497221,-0.529017,NaN,NaN,NaN
19571,2024-06-11 20:00:00-04:00,133.382461,133.392437,132.084866,132.913330,2399900,0.000,1.0,A,1.048050,131.865280,NaN,NaN,NaN,NaN,1.307571,-0.469131,NaN,NaN,NaN
29361,2024-06-12 20:00:00-04:00,132.344372,132.753616,129.799093,130.328125,2555400,0.000,1.0,A,-2.585205,132.913330,NaN,NaN,NaN,NaN,2.954523,-2.016247,NaN,NaN,NaN
39154,2024-06-13 20:00:00-04:00,129.759183,130.497817,129.000596,129.609467,2163300,0.000,1.0,A,-0.718658,130.328125,NaN,NaN,NaN,NaN,1.497221,-0.149716,NaN,NaN,NaN
48951,2024-06-16 20:00:00-04:00,130.867133,131.825361,129.260114,131.585800,2090500,0.000,1.0,A,1.976334,129.609467,NaN,NaN,NaN,NaN,2.565247,0.718667,NaN,NaN,NaN
58754,2024-06-17 20:00:00-04:00,131.885243,134.899638,131.316291,134.650101,3453000,0.000,1.0,A,3.064301,131.585800,NaN,NaN,NaN,NaN,3.583346,2.764858,NaN,NaN,NaN
68560,2024-06-19 20:00:00-04:00,133.013133,134.400557,132.284489,132.484116,2887400,0.000,1.0,A,-2.165985,134.650101,NaN,NaN,NaN,NaN,2.116068,-0.529017,NaN,NaN,NaN
78371,2024-06-20 20:00:00-04:00,132.603906,134.250843,131.955101,133.003159,4461300,0.000,1.0,A,0.519043,132.484116,NaN,NaN,NaN,NaN,2.295742,0.399253,NaN,NaN,NaN
88183,2024-06-23 20:00:00-04:00,135.049367,137.454906,134.091138,134.829773,3339500,0.000,1.0,A,1.826614,133.003159,NaN,NaN,NaN,NaN,3.363768,-0.219594,NaN,NaN,NaN


In [11]:
df = df.dropna(subset=['RSI'])

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 455154 entries, 147150 to 609042
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype                    
---  ------              --------------   -----                    
 0   date                455154 non-null  datetime64[ns, UTC-04:00]
 1   open                455154 non-null  float64                  
 2   high                455154 non-null  float64                  
 3   low                 455154 non-null  float64                  
 4   close               455154 non-null  float64                  
 5   volume              455154 non-null  int64                    
 6   dividends           455154 non-null  float64                  
 7   stock splits        455154 non-null  float64                  
 8   symbol              455154 non-null  object                   
 9   capital gains       455154 non-null  float64                  
 10  previous day close  455154 non-null  float64                  
 11  

In [13]:
# Initialize columns for EMA, MACD, and Signal Line
df['EMA12'] = np.nan
df['EMA26'] = np.nan
df['MACD'] = np.nan
df['SignalLine'] = np.nan

# Group by 'symbol' and calculate EMA, MACD, and Signal Line
for symbol, group in df.groupby('symbol'):
    group['EMA12'] = group['adj close'].ewm(span=12, min_periods=12).mean()
    group['EMA26'] = group['adj close'].ewm(span=26, min_periods=26).mean()
    group['MACD'] = group['EMA12'] - group['EMA26']
    group['SignalLine'] = group['MACD'].ewm(span=9, min_periods=9).mean()

    # Assign back to the original DataFrame
    df.loc[group.index, ['EMA12', 'EMA26', 'MACD', 'SignalLine']] = group[['EMA12', 'EMA26', 'MACD', 'SignalLine']]

# Display the DataFrame with the newly added MACD and Signal Line
print(df.head())

                            date        open        high         low  \
147150 2024-07-01 20:00:00-04:00  127.430000  128.570007  125.440002   
157004 2024-07-02 20:00:00-04:00  126.120003  127.660004  125.830002   
166868 2024-07-04 20:00:00-04:00  126.400002  126.610001  125.290001   
176732 2024-07-07 20:00:00-04:00  126.680000  126.730003  125.400002   
186597 2024-07-08 20:00:00-04:00  126.239998  126.239998  124.160004   

             close   volume  dividends  stock splits symbol  capital gains  \
147150  125.779999  1981100      0.236           1.0      A      -1.384003   
157004  126.129997  1288200      0.000           1.0      A       0.349998   
166868  126.370003  1341700      0.000           1.0      A       0.240005   
176732  126.070000  2091100      0.000           1.0      A      -0.300003   
186597  125.419998  1558900      0.000           1.0      A      -0.650002   

        ...  prev_close   maxdiff   daydiff        RSI         up          dn  \
147150  ...  112.

In [14]:
df.head(50)

,date,open,high,low,close,volume,dividends,stock splits,symbol,capital gains,...,prev_close,maxdiff,daydiff,RSI,up,dn,EMA12,EMA26,MACD,SignalLine
147150,2024-07-01 20:00:00-04:00,127.430000,128.570007,125.440002,125.779999,1981100,0.236,1.0,A,-1.384003,...,112.164001,3.130005,-1.650002,0.000000,60.000000,100.000000,NaN,NaN,NaN,NaN
157004,2024-07-02 20:00:00-04:00,126.120003,127.660004,125.830002,126.129997,1288200,0.000,1.0,A,0.349998,...,111.015999,1.830002,0.009995,23.364366,53.333333,93.333333,NaN,NaN,NaN,NaN
166868,2024-07-04 20:00:00-04:00,126.400002,126.610001,125.290001,126.370003,1341700,0.000,1.0,A,0.240005,...,111.365997,1.320000,-0.029999,33.947164,46.666667,100.000000,NaN,NaN,NaN,NaN
176732,2024-07-07 20:00:00-04:00,126.680000,126.730003,125.400002,126.070000,2091100,0.000,1.0,A,-0.300003,...,111.606003,1.330002,-0.610001,28.950008,40.000000,93.333333,NaN,NaN,NaN,NaN
186597,2024-07-08 20:00:00-04:00,126.239998,126.239998,124.160004,125.419998,1558900,0.000,1.0,A,-0.650002,...,111.306000,2.079994,-0.820000,21.949461,33.333333,100.000000,NaN,NaN,NaN,NaN
196464,2024-07-09 20:00:00-04:00,126.419998,127.139999,125.540001,127.070000,1837300,0.000,1.0,A,1.650002,...,110.655998,1.599998,0.650002,51.636674,26.666667,93.333333,NaN,NaN,NaN,NaN
206333,2024-07-10 20:00:00-04:00,128.800003,131.539993,127.879997,130.820007,1667200,0.000,1.0,A,3.750008,...,112.306000,3.659996,2.020004,74.060312,20.000000,86.666667,NaN,NaN,NaN,NaN
216210,2024-07-11 20:00:00-04:00,131.169998,133.399994,130.639999,132.229996,1477300,0.000,1.0,A,1.409988,...,116.056007,2.759995,1.059998,77.911083,13.333333,80.000000,NaN,NaN,NaN,NaN
226099,2024-07-14 20:00:00-04:00,132.009995,133.139999,130.779999,131.009995,1015700,0.000,1.0,A,-1.220001,...,117.465996,2.360001,-1.000000,69.042682,6.666667,73.333333,NaN,NaN,NaN,NaN
235994,2024-07-15 20:00:00-04:00,131.990005,136.190002,131.600006,136.059998,1064300,0.000,1.0,A,5.050003,...,116.245995,4.589996,4.069992,78.957346,0.000000,66.666667,NaN,NaN,NaN,NaN


In [15]:
# Let's create another variable which is the difference between previous day's Adj Close and current day's Adj Close
df['prev_diff'] = df['adj close'] - df['prev_close']

In [16]:
df.head(50)


,date,open,high,low,close,volume,dividends,stock splits,symbol,capital gains,...,maxdiff,daydiff,RSI,up,dn,EMA12,EMA26,MACD,SignalLine,prev_diff
147150,2024-07-01 20:00:00-04:00,127.430000,128.570007,125.440002,125.779999,1981100,0.236,1.0,A,-1.384003,...,3.130005,-1.650002,0.000000,60.000000,100.000000,NaN,NaN,NaN,NaN,-1.148003
157004,2024-07-02 20:00:00-04:00,126.120003,127.660004,125.830002,126.129997,1288200,0.000,1.0,A,0.349998,...,1.830002,0.009995,23.364366,53.333333,93.333333,NaN,NaN,NaN,NaN,0.349998
166868,2024-07-04 20:00:00-04:00,126.400002,126.610001,125.290001,126.370003,1341700,0.000,1.0,A,0.240005,...,1.320000,-0.029999,33.947164,46.666667,100.000000,NaN,NaN,NaN,NaN,0.240005
176732,2024-07-07 20:00:00-04:00,126.680000,126.730003,125.400002,126.070000,2091100,0.000,1.0,A,-0.300003,...,1.330002,-0.610001,28.950008,40.000000,93.333333,NaN,NaN,NaN,NaN,-0.300003
186597,2024-07-08 20:00:00-04:00,126.239998,126.239998,124.160004,125.419998,1558900,0.000,1.0,A,-0.650002,...,2.079994,-0.820000,21.949461,33.333333,100.000000,NaN,NaN,NaN,NaN,-0.650002
196464,2024-07-09 20:00:00-04:00,126.419998,127.139999,125.540001,127.070000,1837300,0.000,1.0,A,1.650002,...,1.599998,0.650002,51.636674,26.666667,93.333333,NaN,NaN,NaN,NaN,1.650002
206333,2024-07-10 20:00:00-04:00,128.800003,131.539993,127.879997,130.820007,1667200,0.000,1.0,A,3.750008,...,3.659996,2.020004,74.060312,20.000000,86.666667,NaN,NaN,NaN,NaN,3.750008
216210,2024-07-11 20:00:00-04:00,131.169998,133.399994,130.639999,132.229996,1477300,0.000,1.0,A,1.409988,...,2.759995,1.059998,77.911083,13.333333,80.000000,NaN,NaN,NaN,NaN,1.409988
226099,2024-07-14 20:00:00-04:00,132.009995,133.139999,130.779999,131.009995,1015700,0.000,1.0,A,-1.220001,...,2.360001,-1.000000,69.042682,6.666667,73.333333,NaN,NaN,NaN,NaN,-1.220001
235994,2024-07-15 20:00:00-04:00,131.990005,136.190002,131.600006,136.059998,1064300,0.000,1.0,A,5.050003,...,4.589996,4.069992,78.957346,0.000000,66.666667,NaN,NaN,NaN,NaN,5.050003


In [17]:
# Percentage change
# Percentage change on Adj Close for tomorrow
df['change_tomorrow'] = df['adj close'].pct_change(-1)
df['change_tomorrow']

147150   -0.003143
157004   -0.002150
166868    0.002695
176732    0.005874
186597   -0.014692
            ...   
575017    0.054534
585046   -0.000457
595073    0.040928
605101    0.061185
609042         NaN
Name: change_tomorrow, Length: 455154, dtype: float64

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 455154 entries, 147150 to 609042
Data columns (total 26 columns):
 #   Column              Non-Null Count   Dtype                    
---  ------              --------------   -----                    
 0   date                455154 non-null  datetime64[ns, UTC-04:00]
 1   open                455154 non-null  float64                  
 2   high                455154 non-null  float64                  
 3   low                 455154 non-null  float64                  
 4   close               455154 non-null  float64                  
 5   volume              455154 non-null  int64                    
 6   dividends           455154 non-null  float64                  
 7   stock splits        455154 non-null  float64                  
 8   symbol              455154 non-null  object                   
 9   capital gains       455154 non-null  float64                  
 10  previous day close  455154 non-null  float64                  
 11  

In [19]:
# we shall multiply by -1
df.change_tomorrow = df.change_tomorrow* -1
df.head()


,date,open,high,low,close,volume,dividends,stock splits,symbol,capital gains,...,daydiff,RSI,up,dn,EMA12,EMA26,MACD,SignalLine,prev_diff,change_tomorrow
147150,2024-07-01 20:00:00-04:00,127.430000,128.570007,125.440002,125.779999,1981100,0.236,1.0,A,-1.384003,...,-1.650002,0.000000,60.000000,100.000000,NaN,NaN,NaN,NaN,-1.148003,0.003143
157004,2024-07-02 20:00:00-04:00,126.120003,127.660004,125.830002,126.129997,1288200,0.000,1.0,A,0.349998,...,0.009995,23.364366,53.333333,93.333333,NaN,NaN,NaN,NaN,0.349998,0.002150
166868,2024-07-04 20:00:00-04:00,126.400002,126.610001,125.290001,126.370003,1341700,0.000,1.0,A,0.240005,...,-0.029999,33.947164,46.666667,100.000000,NaN,NaN,NaN,NaN,0.240005,-0.002695
176732,2024-07-07 20:00:00-04:00,126.680000,126.730003,125.400002,126.070000,2091100,0.000,1.0,A,-0.300003,...,-0.610001,28.950008,40.000000,93.333333,NaN,NaN,NaN,NaN,-0.300003,-0.005874
186597,2024-07-08 20:00:00-04:00,126.239998,126.239998,124.160004,125.419998,1558900,0.000,1.0,A,-0.650002,...,-0.820000,21.949461,33.333333,100.000000,NaN,NaN,NaN,NaN,-0.650002,0.014692


In [20]:
df.change_tomorrow = df.change_tomorrow* 100
df.head()


,date,open,high,low,close,volume,dividends,stock splits,symbol,capital gains,...,daydiff,RSI,up,dn,EMA12,EMA26,MACD,SignalLine,prev_diff,change_tomorrow
147150,2024-07-01 20:00:00-04:00,127.430000,128.570007,125.440002,125.779999,1981100,0.236,1.0,A,-1.384003,...,-1.650002,0.000000,60.000000,100.000000,NaN,NaN,NaN,NaN,-1.148003,0.314278
157004,2024-07-02 20:00:00-04:00,126.120003,127.660004,125.830002,126.129997,1288200,0.000,1.0,A,0.349998,...,0.009995,23.364366,53.333333,93.333333,NaN,NaN,NaN,NaN,0.349998,0.215047
166868,2024-07-04 20:00:00-04:00,126.400002,126.610001,125.290001,126.370003,1341700,0.000,1.0,A,0.240005,...,-0.029999,33.947164,46.666667,100.000000,NaN,NaN,NaN,NaN,0.240005,-0.269530
176732,2024-07-07 20:00:00-04:00,126.680000,126.730003,125.400002,126.070000,2091100,0.000,1.0,A,-0.300003,...,-0.610001,28.950008,40.000000,93.333333,NaN,NaN,NaN,NaN,-0.300003,-0.587407
186597,2024-07-08 20:00:00-04:00,126.239998,126.239998,124.160004,125.419998,1558900,0.000,1.0,A,-0.650002,...,-0.820000,21.949461,33.333333,100.000000,NaN,NaN,NaN,NaN,-0.650002,1.469202


In [21]:
# Loop through all columns and interpolate missing values
for col in df.columns:
    df[col] = df[col].interpolate(method='linear')  # Interpolate missing values linearly

# Fill remaining NaN values for each column by subtracting the minimum value in each column
for col in df.columns:
    df[col].fillna(df[col].min(), inplace=True)  # Fill remaining NaNs with column's minimum


C:\Users\91628\AppData\Local\Temp\ipykernel_9568\4262225468.py:3: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df[col] = df[col].interpolate(method='linear')  # Interpolate missing values linearly


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 455154 entries, 147150 to 609042
Data columns (total 26 columns):
 #   Column              Non-Null Count   Dtype                    
---  ------              --------------   -----                    
 0   date                455154 non-null  datetime64[ns, UTC-04:00]
 1   open                455154 non-null  float64                  
 2   high                455154 non-null  float64                  
 3   low                 455154 non-null  float64                  
 4   close               455154 non-null  float64                  
 5   volume              455154 non-null  int64                    
 6   dividends           455154 non-null  float64                  
 7   stock splits        455154 non-null  float64                  
 8   symbol              455154 non-null  object                   
 9   capital gains       455154 non-null  float64                  
 10  previous day close  455154 non-null  float64                  
 11  

In [23]:
# change the sign 
 #to know the srock goes up or down
df.change_tomorrow= np.where(df.change_tomorrow>0, 1, -1)
df.head()

,date,open,high,low,close,volume,dividends,stock splits,symbol,capital gains,...,daydiff,RSI,up,dn,EMA12,EMA26,MACD,SignalLine,prev_diff,change_tomorrow
147150,2024-07-01 20:00:00-04:00,127.430000,128.570007,125.440002,125.779999,1981100,0.236,1.0,A,-1.384003,...,-1.650002,0.000000,60.000000,100.000000,-14.98974,-14.988942,-121.455158,-80.883049,-1.148003,1
157004,2024-07-02 20:00:00-04:00,126.120003,127.660004,125.830002,126.129997,1288200,0.000,1.0,A,0.349998,...,0.009995,23.364366,53.333333,93.333333,-14.98974,-14.988942,-121.455158,-80.883049,0.349998,1
166868,2024-07-04 20:00:00-04:00,126.400002,126.610001,125.290001,126.370003,1341700,0.000,1.0,A,0.240005,...,-0.029999,33.947164,46.666667,100.000000,-14.98974,-14.988942,-121.455158,-80.883049,0.240005,-1
176732,2024-07-07 20:00:00-04:00,126.680000,126.730003,125.400002,126.070000,2091100,0.000,1.0,A,-0.300003,...,-0.610001,28.950008,40.000000,93.333333,-14.98974,-14.988942,-121.455158,-80.883049,-0.300003,-1
186597,2024-07-08 20:00:00-04:00,126.239998,126.239998,124.160004,125.419998,1558900,0.000,1.0,A,-0.650002,...,-0.820000,21.949461,33.333333,100.000000,-14.98974,-14.988942,-121.455158,-80.883049,-0.650002,1


In [24]:
# Save the DataFrame as a CSV file
df.to_csv('Feature_engineering.csv', index=False)
